In [1]:
import pandas as pd
import numpy as np
import json
from scipy.sparse import csr_matrix

# [Spotify MPD] Item to Item Collabortive Filtering

## Building the Similarity Matrix

We will primarily use the Item to Item Collaborative Filtering (CF) Algorithim to generate reccomendations. At the high level, the Item to Item CF simply creates a large matrix (S) that holds the pairwise similairties between each item of a data. In our case each item is a song, identified by it's unique track id. So if there a total of 34443 songs in the dataset, S will be a 34443 x 34443 matrix where $S_{ij}$ holds the pairwise similarity betewen song $i$ and song $j$. We calculate similarity using the [Jaccard Index](https://en.wikipedia.org/wiki/Jaccard_index):
$$ S_{ij} = \frac{|\{ p_i\} \cap \{p_j\}|}{|\{p_i\} \cup \{p_j\}|}$$

$\{p_k\}$ here is a **set** of all playlist ids that contain song k. 

Let's start building our similarity matrix (S). We'll begin by loading in the first 1000 playlist splice of our dataset. 

In [2]:
#Read in json splice and transform into a playlist level dataframe
start = 0
end = 1000
path = 'data/mpd.slice.' + str(start) + "-" + str(end-1) + '.json'
d = json.load(open(path, 'r'))
thisSlice = pd.DataFrame.from_dict(d['playlists'], orient='columns')

In [3]:
#Turn playlist level dataframe into song level dataframe
songPlaylistArray = []
for index, row in thisSlice.iterrows():
    for track in row['tracks']:
        songPlaylistArray.append([track['track_uri'], track['artist_name'], track['track_name'], row['pid']])
songPlaylist = pd.DataFrame(songPlaylistArray, columns=['trackid', 'artist_name', 'track_name', 'pid'])

print(songPlaylist.shape)
songPlaylist.head(10)   #is a df of all track ids, cooresponding artist names, track names and playlist ids

(67503, 4)


,trackid,artist_name,track_name,pid
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),0
1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,Toxic,0
2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,Crazy In Love,0
3,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,Rock Your Body,0
4,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,It Wasn't Me,0
5,spotify:track:0XUfyU2QviPAs6bxSpXYG4,Usher,Yeah!,0
6,spotify:track:68vgtRHr7iZHpzGpon6Jlo,Usher,My Boo,0
7,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,The Pussycat Dolls,Buttons,0
8,spotify:track:7H6ev70Weq6DdpZyyTmUXk,Destiny's Child,Say My Name,0
9,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,OutKast,Hey Ya! - Radio Mix / Club Mix,0


Now in order to create S, we could naively look up the playlists that contain song $i$ and song $j$ for each entry in the matrix but that would use a nested for loop and take hours to run. Instead, we'll do some matrix manipulation to obtain the same result. We will start off by building a simlple [co-occurance matrix](https://blogs.msdn.microsoft.com/carlnol/2012/06/23/co-occurrence-approach-to-an-item-based-recommender/). 

Each entry in the co-occurance matrix (C) will hold the number of playlists in which song $i$ and song $j$ are both present. Note, this is exactly the numerator of our similarity metric above. So matrix C will hold all the numerators required to create S, our final similarity matrix. 

In [4]:
## index: pids, columns: unique songs, 0/1 whether its in the playlist
A = songPlaylist.drop(['artist_name', 'track_name'], axis=1).pivot_table(index=["pid"], columns="trackid", aggfunc=lambda x: 1, fill_value=0)
print(A.shape)
A.head()

(1000, 34443)


trackid,spotify:track:000mA0etY38nKdvf1N04af,spotify:track:000xQL6tZNLJzIrtIgxqSl,spotify:track:006AVH7fq061voGXkUiII4,spotify:track:006PJvsr6CyV3JdBf7wiNF,spotify:track:006yrnQMCZpiUgkR612gC8,spotify:track:00BuKLSAFkaEkaVAgIMbeA,spotify:track:00Ci0EXS4fNPnkTbS6wkOh,spotify:track:00CmjeeHvAVKvx3tcIiZTy,spotify:track:00DYRuYJQzfI6dH4Adkimo,spotify:track:00DlEKhhlQNtjnJk7xqB9O,...,spotify:track:7znZvX0Mt6NBmaI8VCPurT,spotify:track:7zo8XAMYBG6nGpqGiIudBc,spotify:track:7zprfY9KDG8g3S5BIOB7xZ,spotify:track:7zqLBFKCBkk5IfbgKgH4VZ,spotify:track:7zscdQe9CjzXnqT3P1Ey7K,spotify:track:7zsw78LtXUD7JfEwH64HK2,spotify:track:7zuwaenG5AF0vG7o7kMduX,spotify:track:7zxRMhXxJMQCeDDg0rKAVo,spotify:track:7zz1drChhd4hQBiGSnLRBZ,spotify:track:7zzBEZBTJejWeL6EqWmCD9
pid,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We can compute the co-occurance matrix (C) using smone matrix manipulation. Note that C is simply $ C = A^T A $.

In [5]:
%%time
A = csr_matrix(A) #scipy's sparse library will compress our array's and speed up computation time

C = A.T * A
print(C.shape)

(34443, 34443)
CPU times: user 1.13 s, sys: 43.7 ms, total: 1.17 s
Wall time: 1.35 s


In [6]:
C.toarray()

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 4, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int64)

Great, now C holds all the numerators we need to create S.

We will create a seperate matrix D to hold our denominator values. Each $D_{ij}$ should hold the length of the union of $\{p_i\}$ and $\{p_j\}$. Se theory tells us $A \cup B = A + B - (A \cap B)$. We can apply length function to both sides of this equation to obtain $|A \cup B| = |A| + |B| - |(A \cap B)|$. Note that $|(A \cap B)|$ is just our co-occurance matrix C. 

So to build D, we will compute D = $(\{p_i\} + \{p_j\}) - C$ for all $i$, $j$.

In [9]:
song_freq_vec = np.sum(A, axis=0)
song_freq_vec = song_freq_vec.reshape(34443, 1)


In [10]:
%%time
C1 = C.toarray()
D  = (song_freq_vec + song_freq_vec.T)-C1

CPU times: user 59.1 s, sys: 45.7 s, total: 1min 44s
Wall time: 2min 24s


Finally, to build S we have C which holds all numerators and D which holds all denominators. To obtain S, sll we need to do is a pairwise division of S by D.

In [11]:
%%time
S1 = np.divide(C1, D)
print(S.shape)
S

CPU times: user 56.6 s, sys: 29.9 s, total: 1min 26s
Wall time: 1min 40s


## Generating Similar Reccomendations

With our similarity matrix (S) at hand, we will generate reccomendations based on which ever songs are most similar to the songs in our testing playlist. For each test playlist, we will splice S to create a new matrix that contians only the songs in the testing playlist in the row index and contians all songs in the column index. Then to compute the similarity a song has to the test playlist all we need to do is add along the columns (or collapse rows). Then for each song will have a similarity metric to the whole test playlist. To generate reccomendations we select the songs that are most similary or have largest sum. 

In [36]:
test = [1, 4, 10, 22, 188, 97, 998]
cooc_2.iloc[test]

trackid,spotify:track:000mA0etY38nKdvf1N04af,spotify:track:000xQL6tZNLJzIrtIgxqSl,spotify:track:006AVH7fq061voGXkUiII4,spotify:track:006PJvsr6CyV3JdBf7wiNF,spotify:track:006yrnQMCZpiUgkR612gC8,spotify:track:00BuKLSAFkaEkaVAgIMbeA,spotify:track:00Ci0EXS4fNPnkTbS6wkOh,spotify:track:00CmjeeHvAVKvx3tcIiZTy,spotify:track:00DYRuYJQzfI6dH4Adkimo,spotify:track:00DlEKhhlQNtjnJk7xqB9O,...,spotify:track:7znZvX0Mt6NBmaI8VCPurT,spotify:track:7zo8XAMYBG6nGpqGiIudBc,spotify:track:7zprfY9KDG8g3S5BIOB7xZ,spotify:track:7zqLBFKCBkk5IfbgKgH4VZ,spotify:track:7zscdQe9CjzXnqT3P1Ey7K,spotify:track:7zsw78LtXUD7JfEwH64HK2,spotify:track:7zuwaenG5AF0vG7o7kMduX,spotify:track:7zxRMhXxJMQCeDDg0rKAVo,spotify:track:7zz1drChhd4hQBiGSnLRBZ,spotify:track:7zzBEZBTJejWeL6EqWmCD9
trackid,,,,,,,,,,,,,,,,,,,,,
spotify:track:000xQL6tZNLJzIrtIgxqSl,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
spotify:track:006yrnQMCZpiUgkR612gC8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
spotify:track:00EhzXcoejkD5RWoS8L4IX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
spotify:track:00W5ThRIn0d1bQNOp0S0af,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
spotify:track:02WDuAXXKumemrvsmrVBvH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
spotify:track:01PLa3LP9WfuAKr3mBRnY4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
spotify:track:0DIekYoiwwr1kkTUGmQlSL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
np.sum(cooc_2.iloc[test], axis=0).describe()

count    34443.000000
mean         0.015455
std          0.108496
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
dtype: float64

In [22]:
""""
def f(df):
    df = df.drop(['artist_name', 'track_name'], axis=1)
    keys,values=df.sort_values('trackid').values.T
    ukeys,index=np.unique(keys,True)
    arrays=np.split(values,index[1:])
    df2=pd.DataFrame({'trackid':ukeys,'pid':[list(a) for a in arrays]})
    return df2
f(songPlaylist)
""""

In [ ]:
# # LOADING ALL 1,000,000 playlists
# songPlaylistArray = []
# start = 0
# end = 1000
# while end != 1000000:
#     path = 'data/mpd.slice.' + str(start) + "-" + str(end-1) + '.json'
#     start += 1000
#     end += 1000
#     d = json.load(open(path, 'r'))
#     thisSlice = pd.DataFrame.from_dict(d['playlists'], orient='columns')
#     for index, row in thisSlice.iterrows():
#         for track in row['tracks']:
#             songPlaylistArray.append([track['track_uri'], track['artist_name'], track['track_name'], row['pid']])
# songPlaylist = pd.DataFrame(songPlaylistArray, columns=['trackid', 'artist_name', 'track_name', 'pid'])